## Notebook to visualize omics QTL and colocalization with PD risk tracks for the FOUNDIN-PD QTL project

will start with single figure with localized tracks for each regions including GWAS, gene annotations, 'risk' ATAC peaks, colocalization H4 probilities for different omics features and then QTL tracks , will also include Encode CRE, then show try to incorporate the Hi-C data and SCAT co-asscessibility 

using pyGenomeTracks to generate the figure
- [pyGenomeTracks github](https://pygenometracks.readthedocs.io/en/latest/index.html)
- [pyGenomeTrack docs](https://pygenometracks.readthedocs.io/en/latest/index.html)

In [ ]:
!date

#### import libraries

In [ ]:
from pandas import read_parquet, read_csv, DataFrame, read_hdf, concat
from math import ceil, floor
from numpy import log10
from IPython.display import Image
from os.path import exists

#### set notebook variables

In [ ]:
# parameters
index_variant = 'rs13294100'
day = 'daNA'
modality = 'DAn-meta'

In [ ]:
# naming
cohort = 'foundin'
dx = 'PD'
set_name = f'{cohort}_{day}_{modality}_{dx}'

# directories
wrk_dir = '/home/jupyter/foundin_qtl'
quants_dir = f'{wrk_dir}/quants'
results_dir = f'{wrk_dir}/results'
public_dir = f'{wrk_dir}/public'
figures_dir = f'{wrk_dir}/figures'

# in files
gwas_sum_stats_file = f'{public_dir}/nalls_pd_gwas/pd_sumstats_23andme_buildGRCh38.tsv.gz'
# gwas_sum_stats_file = f'{public_dir}/nalls_pd_gwas/pd_sumstats_no23andme_buildGRCh38.tsv.gz'
gene_annot_file = f'{public_dir}/hg38.ncbiRefSeq.gtf.gz'
encode_bb_file = f'{public_dir}/encodeCcreCombined.bb'
coloc_file = f'{results_dir}/{set_name}.coloc.pp.csv'
causals_file = f'{results_dir}/{set_name}.casuals.pp.parquet'

# out files
figure_file = f'{figures_dir}/{cohort}.{index_variant}.{dx}.locus.png'
config_file = f'{figures_dir}/tracks/{cohort}.{index_variant}.{dx}.locus.ini'
gwas_bedgraph_file = f'{figures_dir}/tracks/{dx}.gwas.{index_variant}.bedgraph'
encode_bed_file = f'{figures_dir}/tracks/{dx}.{index_variant}.encodeCcreCombined.bed'

# variables
DEBUG = False
config_content = ''
nl = '\n'
HIGH_LD = 0.8
MAX_DIST = 1000000
FLANK_PAD = 5000
SUG_THRESHOLD = 1.00e-05
MIN_CICERO = 0.2
min_h4 = 0.2
days = ['da0', 'da25', 'da65']
day_colors = {'da0': 'black', 'da25': 'green', 'da65': 'blue'}

#### utility functions

In [ ]:
# run command line procsss with bash majic
# make this a small function so it can be target function for parallelization if needed
def run_bash_cmd(this_cmd: str, verbose: bool=False):
    if verbose:
        print(this_cmd)
    !{this_cmd}

### GWAS track

#### load the summary stats

In [ ]:
%%time
gwas_df = read_csv(gwas_sum_stats_file, sep='\t')
print(gwas_df.shape)
if DEBUG:
    display(gwas_df.sample(5))

#### set the actual edges based on statistically significant p-values for region centered on index variant

In [ ]:
risk_index = gwas_df.loc[gwas_df.variant_id == index_variant]
chrom = risk_index.chromosome.values[0]
start_bp = risk_index.base_pair_location.values[0] - MAX_DIST
stop_bp = risk_index.base_pair_location.values[0] + MAX_DIST
print(f'initial region is chrom{chrom}:{start_bp}-{stop_bp}')

In [ ]:
# filter to the specified region
gwas_df = gwas_df.loc[(gwas_df.chromosome == chrom) & 
                      (gwas_df.base_pair_location >= start_bp) & 
                      (gwas_df.base_pair_location <= stop_bp)]
risk_df = gwas_df.loc[(gwas_df.p_value <= SUG_THRESHOLD)]
start_bp = risk_df.base_pair_location.min() - FLANK_PAD
stop_bp = risk_df.base_pair_location.max() + FLANK_PAD
print(f'risk region is chrom{chrom}:{start_bp}-{stop_bp}')
print(f'risk region shape {risk_df.shape}')
if DEBUG:
    display(risk_df.head())

#### convert gwas df to bedgraph

In [ ]:
# re-filter with narrower edges
gwas_df = gwas_df.loc[(gwas_df.chromosome == chrom) & 
                      (gwas_df.base_pair_location >= start_bp) & 
                      (gwas_df.base_pair_location <= stop_bp)]
gwas_bed = gwas_df[['chromosome', 'base_pair_location', 'variant_id']].copy()
gwas_bed.rename(columns={'chromosome': 'chr', 'base_pair_location': 'position', 
                         'variant_id': 'id'}, inplace=True)
gwas_bed['end'] = gwas_bed['position'] + 1
gwas_bed['signal'] = -log10(gwas_df.p_value)
gwas_bed = gwas_bed[['chr', 'position', 'end', 'signal', 'id']]
print(gwas_bed.shape)
if DEBUG:
    display(gwas_bed.sample(5))
# save the gwas bedgraph
gwas_bed.to_csv(gwas_bedgraph_file, index=False, header=False, sep='\t')

#### format gwas track config string

In [ ]:
this_content = f'[spacer]{nl}[GWAS]{nl}\
title = {dx} GWAS{nl}\
file = {gwas_bedgraph_file}{nl}\
file_type = bedgraph{nl}\
type = points:6{nl}\
color = black{nl}\
use_middle = true{nl}\
min_value = 0{nl}\
max_value = {ceil(gwas_bed.signal.max())}{nl}\
height = 3{nl}{nl}\
[hlines ovelayed]{nl}\
color = grey{nl}\
line_width = 1{nl}\
line_style = dotted{nl}\
y_values = 7.3{nl}\
file_type = hlines{nl}\
overlay_previous = share-y{nl}\
[x-axis]{nl}{nl}'

config_content = config_content + this_content

if DEBUG:
    print(this_content)

### format the genes annot config string

In [ ]:
this_content = f'{nl}{nl}\
[genes]{nl}\
title = Genes{nl}\
file = {gene_annot_file}{nl}\
height = 2{nl}\
color = darkblue{nl}\
merge_transcripts = true{nl}\
prefered_name = gene_name{nl}\
fontsize = 12{nl}\
style = UCSC{nl}\
max_labels = 200{nl}\
file_type = gtf{nl}{nl}'

config_content = config_content + this_content

if DEBUG:
    print(this_content)

### eQTL colocalization track

#### load the colocalization feature probabilities

In [ ]:
coloc_df = read_csv(coloc_file)
print(f'colocalization shape {coloc_df.shape}')
# subset based on min
coloc_df = coloc_df.loc[coloc_df.H4 > min_h4]
print(f'colocalization shape at min_h4 of {min_h4}: {coloc_df.shape}')
if DEBUG:
    display(coloc_df)

#### load the colocalization variant probabilities

In [ ]:
def format_qtl_bedgraph(df: DataFrame, out_file: str, verbose: bool=True) -> float:
    qtl_bed = df[['chromosome', 'base_pair_location']].copy()
    qtl_bed['variant_id'] = df.index.values
    qtl_bed['end'] = qtl_bed['base_pair_location'] + 1
    qtl_bed['signal'] = -log10(df.p_value_qtl)
    # qtl_bed['signal'] = df.logABF_qtl    
    qtl_bed = qtl_bed[['chromosome', 'base_pair_location', 'end', 'signal', 'variant_id']]
    print(qtl_bed.shape)
    if verbose:
        display(qtl_bed.head())    
    qtl_bed.to_csv(out_file, index=False, header=False, sep='\t')
    return qtl_bed.signal.max()

def format_pip_bedgraph(df: DataFrame, feature: str, verbose: bool=True) -> float:
    qtl_bed = df[['chromosome', 'base_pair_location']].copy()
    qtl_bed['variant_id'] = df.index.values
    qtl_bed['end'] = qtl_bed['base_pair_location'] + 1
    qtl_bed['signal'] = -log10(df.p_value_qtl)
    qtl_bed = qtl_bed[['chromosome', 'base_pair_location', 'end', 'signal', 'variant_id']]
    for pp_type in ['risk', 'qtl', 'h4']:
        qtl_bed.signal = df[[f'{pp_type}_PiP']]
        this_file = f'{figures_dir}/tracks/{dx}.{feature}_{pp_type}_pip.{index_variant}.bedgraph'
        qtl_bed.to_csv(this_file, index=False, header=False, sep='\t') 
        print(f'{pp_type} bed shape {qtl_bed.shape}')
        print(f'{pp_type} bed max {qtl_bed.signal.max()}')

In [ ]:
qtl_max = 0
causals_df = read_parquet(causals_file)
# rename PIPs for ease of use later
causals_df = causals_df.rename(columns={'PP_risk': 'risk_PiP', 
                                        'PP_qtl': 'qtl_PiP', 
                                        'h4_pp': 'h4_PiP'})
print(f'causals shape {causals_df.shape}')
index_subset = causals_df.loc[causals_df.index == index_variant]
print(f'index variant causals shape {index_subset.shape}')
features = index_subset.MarkerName.str.split(':', expand=True)[0].unique()
# only include those above min_H4
features = list(set(features) & set(coloc_df.feature))
for feature in features:
    print(feature)
    feature_causals_df = causals_df.loc[causals_df.MarkerName.str.startswith(f'{feature}:')]
    print(f'feature causals shape {feature_causals_df.shape}')
    qtl_out_file = f'{figures_dir}/tracks/{dx}.{feature}.qtl.{index_variant}.bedgraph'
    feature_max = format_qtl_bedgraph(feature_causals_df, qtl_out_file, verbose=DEBUG)
    format_pip_bedgraph(feature_causals_df, feature, verbose=DEBUG)
    if feature_max > qtl_max:
        qtl_max = feature_max
    if DEBUG:
        display(feature_causals_df.head())

#### format qtl tracks config string

In [ ]:
for feature in features:
    qtl_bedgraph_file = f'{figures_dir}/tracks/{dx}.{feature}.qtl.{index_variant}.bedgraph'
    this_content = f'[eQTL]{nl}\
    title = {feature}{nl}\
        DAn meta-eQTL{nl}\
    file = {qtl_bedgraph_file}{nl}\
    file_type = bedgraph{nl}\
    type = points:8{nl}\
    color = black{nl}\
    use_middle = true{nl}\
    max_value = {ceil(qtl_max+1)}{nl}\
    height = 3{nl}\
    [hlines ovelayed]{nl}\
    color = black{nl}\
    line_width = 0.5{nl}\
    line_style = solid{nl}\
    y_values = 0{nl}\
    file_type = hlines{nl}\
    overlay_previous = share-y{nl}\
    {nl}{nl}'    
    
    # min_value = 0{nl}\    

    config_content = config_content + this_content

    if DEBUG:
        print(this_content)

#### format risk and qtl colocalization PiP tracks config string

In [ ]:
# for feature in features:
#     this_file = f'{figures_dir}/tracks/{dx}.{feature}_risk_pip.{index_variant}.bedgraph'
#     this_content = f'[PiPs]{nl}\
#     title = {modality} PiPs{nl}\
#         {dx} (black){nl}\
#         {feature} (purple){nl}\
#         H4 (red){nl}\
#     file = {this_file}{nl}\
#     file_type = bedgraph{nl}\
#     type = points:8{nl}\
#     color = black{nl}\
#     use_middle = true{nl}\
#     min_value = 0{nl}\
#     max_value = 1.1{nl}\
#     height = 2{nl}{nl}'

#     if exists(this_file):
#         config_content = config_content + this_content

#     if DEBUG:
#         print(this_content)

#     this_file = f'{figures_dir}/tracks/{dx}.{feature}_qtl_pip.{index_variant}.bedgraph'    
#     this_content = f'[PiPs]{nl}\
#     file = {this_file}{nl}\
#     file_type = bedgraph{nl}\
#     type = points:8{nl}\
#     color = purple{nl}\
#     use_middle = true{nl}\
#     overlay_previous = share-y{nl}{nl}'

#     if exists(this_file):
#         config_content = config_content + this_content
    
#     this_file = f'{figures_dir}/tracks/{dx}.{feature}_h4_pip.{index_variant}.bedgraph'
#     this_content = f'[PiPs]{nl}\
#     file = {this_file}{nl}\
#     file_type = bedgraph{nl}\
#     type = points:8{nl}\
#     color = red{nl}\
#     use_middle = true{nl}\
#     overlay_previous = share-y{nl}\
#     [x-axis]{nl}{nl}'

#     if exists(this_file):
#         config_content = config_content + this_content

#     if DEBUG:
#         print(this_content)        

### format the SCAT peaks

In [ ]:
# this is from Frank's formatting and files
this_content = f'[Dopaminergic_Neurons]{nl}\
file = {figures_dir}/tracks/scat_Dopaminergic_Neurons_means.bedgraph{nl}\
color = #FFD700{nl}\
height = 1.5{nl}\
title = DA{nl}\
min_value = 0.0{nl}\
max_value = 0.70{nl}{nl}\
[Immature_Dopaminergic_Neurons]{nl}\
file = {figures_dir}/tracks/scat_Immature_Dopaminergic_Neurons_means.bedgraph{nl}\
color = #EE5C42{nl}\
height = 1.5{nl}\
title = iDA{nl}\
min_value = 0.0{nl}\
max_value = 0.70{nl}{nl}\
[Late_neuron_Progenitor]{nl}\
file = {figures_dir}/tracks/scat_Late_neuron_Progenitor_means.bedgraph{nl}\
color = #2E8B57{nl}\
height = 1.5{nl}\
title = LNP{nl}\
min_value = 0.0{nl}\
max_value = 0.70{nl}{nl}'

config_content = config_content + this_content

if DEBUG:
    print(this_content)

### SCAT co-accessibility

In [ ]:
def format_cicero_arcs(cicero_df: DataFrame, chrom: str, verbose: bool=False) -> DataFrame:
    # to speed up a bit do initial subset by chrom
    cicero_df = cicero_df.loc[cicero_df[0].str.startswith(f'chr{chrom}-')]
    cicero_score_df = concat([cicero_df[0].str.split(r':|-', expand=True), 
                              cicero_df[1].str.split(r':|-', expand=True), 
                              cicero_df[[2]]], axis=1)
    cicero_score_df.columns=['chrom1', 'start1', 'stop1', 'chrom2', 
                             'start2', 'stop2', 'coaccess']
    cicero_score_df[['start1', 'stop1', 
                     'start2', 'stop2']]=cicero_score_df[['start1', 'stop1', 
                                                          'start2', 'stop2']].astype('int')
    print(cicero_score_df.shape) 
    # convert coaccess df to arcs
    cicero_arcs_df = cicero_score_df.loc[(cicero_score_df.chrom1 == f'chr{chrom}') & 
                          (cicero_score_df.start1 >= start_bp) & 
                          (cicero_score_df.stop1 <= stop_bp) & 
                          (cicero_score_df['coaccess'].notna()) &
                          (cicero_score_df.coaccess > MIN_CICERO)]
    print(cicero_arcs_df.shape)
    return cicero_arcs_df

#### load and format the Cicero results by cell-type

In [ ]:
%%time
# here not including all but most relevant
# scat_cell_types = ['Dopaminergic_Neurons', 'Immature_Dopaminergic_Neurons', 'Late_neuron_Progenitor']
scat_cell_types = ['Dopaminergic_Neurons', 'Immature_Dopaminergic_Neurons']
max_cicero_value = 0
for cell_type in scat_cell_types:
    print(cell_type)
    cicero_in_file = f'{quants_dir}/SCAT.{cell_type}.cicero.conns.txt.gz'
    cicero_out_file = f'{figures_dir}/tracks/{dx}.{cell_type}.cicero.{index_variant}.arcs'
    cicero_df = read_csv(cicero_in_file, sep='\t',skiprows=1, header=None)
    print(cicero_df.shape)
    cicero_arcs_df = format_cicero_arcs(cicero_df, chrom, verbose=DEBUG)
    this_max = cicero_arcs_df.coaccess.max()
    print(this_max)
    if this_max > max_cicero_value:
        max_cicero_value = this_max 
    cicero_arcs_df.to_csv(cicero_out_file, index=False, header=False, sep='\t')        
    if DEBUG:
        display(cicero_df.sample(5))    

### format cicero SCAT track config string

In [ ]:
# scat_colors = {'Dopaminergic_Neurons': '#FFD700', 'Immature_Dopaminergic_Neurons': '#EE5C42', 
#                'Late_neuron_Progenitor': '#2E8B57', 'Early_neuron_Progenitor': '#4682B4', 
#                'Proliferating_Floor_Plate_Progenitors': '#8B8B00'}
scat_colors = {'Dopaminergic_Neurons': 'Blues', 
               'Immature_Dopaminergic_Neurons': 'Reds', 
               'Late_neuron_Progenitor': 'Greens'}
for cell_type in scat_cell_types:
    cicero_arcs_file = f'{figures_dir}/tracks/{dx}.{cell_type}.cicero.{index_variant}.arcs'
    this_color = scat_colors.get(cell_type)
    this_content = f'[Cierco]{nl}\
title = {cell_type}{nl}\
    co-accessibility{nl}\
file = {cicero_arcs_file}{nl}\
file_type = links{nl}\
color = {this_color}{nl}\
min_value = -0.2{nl}\
max_value = {ceil(max_cicero_value)}{nl}\
height = 4{nl}\
line_type = arcs{nl}\
line_width = 0.5{nl}\
orientation = inverted{nl}\
compact_arcs_level = 1{nl}{nl}'

    config_content = config_content + this_content

    if DEBUG:
        print(this_content)

### caQTL colocalization scores track

In [ ]:
features_file = f'{quants_dir}/{cohort}_consensus_peaks.saf'
features_df = read_csv(features_file, sep='\t')
features_df.columns = ['feature', 'chrom', 'start', 'end', 'strand']
print(f'features annot shape {features_df.shape}')
if DEBUG:
    display(features_df.head())

In [ ]:
def format_chst_qtl_bedgraph(features_df: DataFrame, day: str, chst_modality: str, 
                             verbose: bool=False):
    this_file = f'{results_dir}/{cohort}_{day}_{chst_modality}_{dx}.coloc.pp.csv'
    coloc_df = read_csv(this_file)
    print(f'{day} {chst_modality} coloc shape {coloc_df.shape}')
    coloc_df = coloc_df.merge(features_df, how='left', on='feature')
    print(f'annotated {day} {chst_modality} coloc shape {coloc_df.shape}')
    # format the chrom
    coloc_df.chrom = coloc_df.chrom.str.replace('chr', '')
    # subset to region
    coloc_df = coloc_df.loc[(coloc_df.chrom == chrom) & 
                            (coloc_df.start >= start_bp) & 
                            (coloc_df.end <= stop_bp)]
    print(f'risk region {day} {chst_modality} coloc shape {coloc_df.shape}')
    qtl_bed = coloc_df[['chrom', 'start', 'end', 'H4', 'feature']]
    bedgraph_file = f'{figures_dir}/tracks/{dx}.{day}_{chst_modality}.qtl_h4.{index_variant}.bedgraph'
    qtl_bed.to_csv(bedgraph_file, index=False, header=False, sep='\t')        
    if DEBUG:
        display(coloc_df.head())
        display(qtl_bed.head())

In [ ]:
chst_modality = 'ATAC'
for day in days:
    print(day)
    format_chst_qtl_bedgraph(features_df, day, chst_modality, verbose=DEBUG)

#### format caQTL colocalization H4 track config string

In [ ]:
# for day in days:
#     day_color = day_colors.get(day)
#     bedgraph_file = f'{figures_dir}/tracks/{dx}.{day}_{chst_modality}.qtl_h4.{index_variant}.bedgraph'
#     this_content = f'[{day} {chst_modality}]{nl}\
#     file = {bedgraph_file}{nl}\
#     title = {day} caQTL H4{nl}\
#     file_type = bedgraph{nl}\
#     color = {day_color}{nl}\
#     height = 2{nl}\
#     min_value = 0.0{nl}\
#     max_value = 1{nl}\
#     [hlines ovelayed]{nl}\
#     color = black{nl}\
#     line_width = 0.5{nl}\
#     line_style = solid{nl}\
#     y_values = 0{nl}\
#     file_type = hlines{nl}\
#     overlay_previous = share-y{nl}\
#     {nl}{nl}'

#     config_content = config_content + this_content

#     if DEBUG:
#         print(this_content)

### mQTL colocalization scores track

In [ ]:
features_file = f'{quants_dir}/EPIC_annotation_hg38.txt'
features_df = read_csv(features_file, sep='\t', header=None)
features_df.columns = ['chrom', 'start', 'end', 'feature']
print(f'features annot shape {features_df.shape}')
if DEBUG:
    display(features_df.head())

In [ ]:
chst_modality = 'METH'
for day in days:
    print(day)
    if day == 'da25':
        continue
    format_chst_qtl_bedgraph(features_df, day, chst_modality, verbose=DEBUG)

#### format mQTL colocalization H4 tracks config string

In [ ]:
# for day in days:
#     if day == 'da25':
#         continue
#     day_color = day_colors.get(day)
#     bedgraph_file = f'{figures_dir}/tracks/{dx}.{day}_{chst_modality}.qtl_h4.{index_variant}.bedgraph'
#     this_content = f'[{day} {chst_modality}]{nl}\
#     file = {bedgraph_file}{nl}\
#     title = {day} mQTL H4{nl}\
#     file_type = bedgraph{nl}\
#     color = {day_color}{nl}\
#     height = 2{nl}\
#     min_value = 0.0{nl}\
#     max_value = 1{nl}\
#     [hlines ovelayed]{nl}\
#     color = black{nl}\
#     line_width = 0.5{nl}\
#     line_style = solid{nl}\
#     y_values = 0{nl}\
#     file_type = hlines{nl}\
#     overlay_previous = share-y{nl}\
#     {nl}{nl}'

#     config_content = config_content + this_content

#     if DEBUG:
#         print(this_content)

### Bryois CNS cell-type eQTL track

In [ ]:
cns_names = ['Bryois-Astro', 'Bryois-Endo', 'Bryois-ExN', 'Bryois-InN', 
             'Bryois-Micro', 'Bryois-OPC', 'Bryois-Oligo', 'Bryois-Peri']

for cell_type in cns_names:
    print(cell_type)
    cns_file = f'{results_dir}/{cohort}_daNA_{cell_type}_{dx}.casuals.pp.parquet'
    causals_df = read_parquet(cns_file)
    # rename PIPs for ease of use later
    causals_df = causals_df.rename(columns={'PP_risk': 'risk_PiP', 
                                            'PP_qtl': 'qtl_PiP', 
                                            'h4_pp': 'h4_PiP', 
                                            'pval_nominal': 'p_value_qtl'})
    print(f'causals shape {causals_df.shape}')
    # use same features from foundin eQTL
    for feature in features:
        print(feature)
        feature_causals_df = causals_df.loc[causals_df.phenotype_id == feature]
        print(f'feature causals shape {feature_causals_df.shape}')
        qtl_out_file = f'{figures_dir}/tracks/{dx}.{feature}.{cell_type}.qtl.{index_variant}.bedgraph'
        feature_max = format_qtl_bedgraph(feature_causals_df, qtl_out_file, verbose=DEBUG)
        if feature_max > qtl_max:
            qtl_max = feature_max
        if DEBUG:
            display(feature_causals_df.head())    

#### format Bryois CNS qtl track(s) config string

In [ ]:
cns_colors = {'Bryois-Astro': 'blue', 'Bryois-Endo': 'purple', 
              'Bryois-ExN': 'violet', 'Bryois-InN': 'red', 
              'Bryois-Micro': 'green', 'Bryois-OPC': 'orange', 
              'Bryois-Oligo': 'black', 'Bryois-Peri': 'grey'}
for feature in features:
    for index, cell_type in enumerate(cns_names):
        print(cell_type, index)
        cell_color = cns_colors.get(cell_type)
        this_file = f'{figures_dir}/tracks/{dx}.{feature}.{cell_type}.qtl.{index_variant}.bedgraph'
        # specify first cell-type Astro, then loop the rest as overlay
        if index == 0:
            this_content = f'[CNS eQTL]{nl}\
            title = Bryois CNS{nl}\
                {feature} eQTL{nl}\
                Astro (blue){nl}\
                Endo (purple){nl}\
                ExN (violet){nl}\
                InN (red){nl}\
                Micro (green){nl}\
                OPC (orange){nl}\
                Oligo (black){nl}\
                Peri (grey){nl}\
            file = {this_file}{nl}\
            file_type = bedgraph{nl}\
            type = points:8{nl}\
            color = {cell_color}{nl}\
            use_middle = true{nl}\
            min_value = 0{nl}\
            max_value = {ceil(qtl_max+1)}{nl}\
            height = 6{nl}\
            [hlines ovelayed]{nl}\
            color = black{nl}\
            line_width = 0.5{nl}\
            line_style = solid{nl}\
            y_values = 0{nl}\
            file_type = hlines{nl}\
            overlay_previous = share-y{nl}\
            {nl}{nl}'            
        else:
            this_content = f'[CNS eQTL]{nl}\
            file = {this_file}{nl}\
            file_type = bedgraph{nl}\
            type = points:8{nl}\
            color = {cell_color}{nl}\
            use_middle = true{nl}\
            overlay_previous = share-y{nl}\
            [hlines ovelayed]{nl}\
            color = black{nl}\
            line_width = 0.5{nl}\
            line_style = solid{nl}\
            y_values = 0{nl}\
            file_type = hlines{nl}\
            overlay_previous = share-y{nl}\
            {nl}{nl}'            

        config_content = config_content + this_content
        if DEBUG:
            print(this_content)

### create the Encode CRE bed

In [ ]:
this_cmd = f'bigBedToBed {encode_bb_file} -chrom=chr{chrom} -start={start_bp} \
-end={stop_bp} {encode_bed_file}'
run_bash_cmd(this_cmd, DEBUG)

### format the HiC data tracks

In [ ]:
# # this is from Frank's formatting and files
# this_content = f'[HiC regions da0]{nl}\
# file = {figures_dir}/tracks/hic_da0_regions.bed{nl}\
# height = 0.25{nl}\
# title = da0 Regions{nl}\
# file_type = bed{nl}\
# display = collapsed{nl}\
# color = #00AFBB{nl}\
# labels = false{nl}{nl}\
# [HiC da0]{nl}\
# file = {figures_dir}/tracks/hic_da0.bedpe{nl}\
# title = da0 Loops{nl}\
# height = 2{nl}\
# line_width = 2{nl}\
# links_type = arcs{nl}\
# color = #00AFBB{nl}\
# line_style = solid{nl}\
# file_type = links{nl}\
# orientation = inverted{nl}\
# use_middle = true{nl}{nl}\
# [HiC regions da65]{nl}\
# file = {figures_dir}/tracks/hic_da65_regions.bed{nl}\
# height = 0.25{nl}\
# title = da65 Regions{nl}\
# file_type = bed{nl}\
# display = collapsed{nl}\
# color = #FC4E07{nl}\
# labels = false{nl}{nl}\
# [HiC da65]{nl}\
# file = {figures_dir}/tracks/hic_da65.bedpe{nl}\
# title = da65 Loops{nl}\
# height = 2{nl}\
# line_width = 2{nl}\
# links_type = arcs{nl}\
# color = #FC4E07{nl}\
# line_style = solid{nl}\
# file_type = links{nl}\
# orientation = inverted{nl}\
# use_middle = true{nl}{nl}'

# config_content = config_content + this_content

# if DEBUG:
#     print(this_content)

### add encode track to config

In [ ]:
this_content = f'{nl}{nl}\
[CRE]{nl}\
title = ENCODE CRE{nl}\
file = {encode_bed_file}{nl}\
height = 2{nl}\
fontsize = 8{nl}\
file_type = bed{nl}\
color = Blues{nl}\
display = collapsed{nl}\
labels = false{nl}\
border_color = black{nl}\
show_data_range = false{nl}\
[x-axis]{nl}{nl}'

config_content = config_content + this_content

if DEBUG:
    print(this_content)

### save the config file content

In [ ]:
with open(config_file, 'w') as f:
    f.write(config_content)

### generate the figure

In [ ]:
this_cmd = f'pyGenomeTracks --tracks {config_file} \
--region chr{chrom}:{start_bp}-{stop_bp} --dpi 200 \
--title "{dx} {index_variant}" \
--outFileName {figure_file}'

run_bash_cmd(this_cmd, DEBUG)

### view the generated figure

In [ ]:
Image(figure_file)

In [ ]:
!date